In [12]:
!pip install tensorflow

In [13]:
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np
import matplotlib.pyplot as plt


In [14]:
(x_train,_),(x_test,_) = tf.keras.datasets.mnist.load_data()

In [15]:
x_train = x_train/255
x_test = x_test/255

x_train = np.expand_dims(x_train,axis=-1)
x_test = np.expand_dims(x_test,axis=-1)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [20]:
encoder = models.Sequential([
    layers.InputLayer(input_shape=(28,28,1)),
    layers.Conv2D(32,(3,3),activation='relu',strides=2, padding='same'),
    layers.Conv2D(64,(3,3),activation='relu',strides=2, padding='same'),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),

])

decoder = models.Sequential([
    layers.InputLayer(input_shape=(64,)),
    layers.Dense(7*7*64,activation='relu'),
    layers.Reshape((7,7,64)),
    layers.Conv2DTranspose(64,(3,3),strides=2,activation='relu',padding='same'),
    layers.Conv2DTranspose(32,(3,3),strides=2,activation='relu',padding='same'),
    layers.Conv2DTranspose(1,(3,3),activation='sigmoid',padding='same')
])

autoencoder = models.Sequential([
    encoder,
    decoder
])



/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [23]:
print("Encoder Summary")
encoder.summary()
print("Decoder Summary")
decoder.summary()
print("Autoencoder Summary")
autoencoder.summary()

Encoder Summary


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 14, 14, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │       200,768 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 219,584 (857.75 KB)

 Trainable params: 219,584 (857.75 KB)

...

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)       │ (None, 64)             │       219,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (None, 28, 28, 1)      │       259,521 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 479,105 (1.83 MB)

 Trainable params: 479,105 (1.83 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
autoencoder.compile(optimizer='adam',loss='binary_crossentropy')

In [25]:
autoencoder.fit(x_train,x_train,epochs=10,batch_size=128,validation_data=(x_test,x_test))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 110s 228ms/step - loss: 0.2778 - val_loss: 0.0877
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 137s 219ms/step - loss: 0.0849 - val_loss: 0.0772
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 143s 221ms/step - loss: 0.0764 - val_loss: 0.0735
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 141s 220ms/step - loss: 0.0735 - val_loss: 0.0719
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 143s 221ms/step - loss: 0.0720 - val_loss: 0.0711
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 142s 222ms/step - loss: 0.0710 - val_loss: 0.0701
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 141s 219ms/step - loss: 0.0700 - val_loss: 0.0694
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 158s 254ms/step - loss: 0.0695 - val_loss: 0.0687
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 128s 223ms/step - loss: 0.0690 - val_loss: 0.0684
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 107s 227ms/step - loss: 0.0686 - val_loss: 0.0680


In [ ]:
decoded_images = autoencoder.predict(x_test)

n=1000
plt.figure(figsize=(20,4))


for i in range(n):
  ax = plt.subplot(2,n,i+1)
  plt.imshow(x_test[i].reshape(28,28),cmap='gray')
  plt.title("Original")
  plt.gray()
  plt.axis('off')

  ax = plt.subplot(2,n,i+1+n)
  plt.imshow(decoded_images[i].reshape(28,28),cmap='gray')
  plt.title("Decoded")
  plt.gray()
  plt.axis('off')

plt.show()

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step


In [27]:
autoencoder.save('autoencoder_mnist.keras')